In [ ]:
import os
import psycopg2
import logging
import requests
from rawg.db import get_conn, get_last_added, insert_games
from rawg.rawg_api import fetch_new_games
from rawg.storage import save_local, upload_s3

logger = logging.getLogger(__name__)
RAWG_API_KEY = os.getenv('RAWG_API_KEY')
logger.setLevel(logging.INFO)


def get_conn():
    try:
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT', 5432)
        )
        conn.autocommit = False
        return conn
    except psycopg2.OperationalError:
        logger.exception("DB connection failed")
        raise

def get_last_added(conn):
    with conn.cursor() as cur:
        cur.execute("SELECT COALESCE(MAX(added), 0) FROM rawg_games;")
        return cur.fetchone()[0] or 0


def fetch_new_games(last_added, page_size=40):
    page = 1
    new_games = []
    while True:
        try:
            resp = requests.get(
                'https://api.rawg.io/api/games',
                params={'key': RAWG_API_KEY, 'ordering': '-added',
                        'page_size': page_size, 'page': page},
                timeout=10
            )
            resp.raise_for_status()
        except requests.RequestException:
            logger.exception("RAWG API error")
            break

        data = resp.json()
        filtered = [g for g in data.get('results', []) if g.get('added',0) > last_added]
        if not filtered:
            break
        new_games.extend(filtered)
        if not data.get('next'):
            break
        page += 1
    return new_games


def handler(event, context):
    conn = get_conn()
    try:
        last = get_last_added(conn)
        logger.info("Last added in DB: %s", last)

        new = fetch_new_games(last)
        if not new:
            logger.info("No new games.")
            return {'new_count': 0}

        path = save_local(new)
        upload_s3(path)

        count = insert_games(conn, new)
        logger.info("Inserted %d new games", count)
        return {'new_count': count}

    except Exception:
        logger.exception("Handler failed")
        return {'error': True}
    finally:
        conn.close()